In [2]:
import torch 
import numpy as np 
import matplotlib.pyplot as plt 
import timm  # timm (PyTorch Image Models) library for accessing pre-trained models and utilities for image classification
import os  
from tqdm.notebook import tqdm  # tqdm for displaying progress bars in Jupyter notebooks
from torchvision import transforms as T, datasets
from helper import view_classify, show_image, show_grid, accuracy # Importing custom helper functions for visualization and calculating accuracy

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from torch import nn 
import torch.nn.functional as F

# Creating a pre-trained model from timm (PyTorch Image Models)
model_efficientnet = timm.create_model('tf_efficientnet_b4_ns', pretrained=False)  # Load a pre-trained model specified in CFG

# Customizing the classifier part of the model
model_efficientnet.classifier = nn.Sequential(
    nn.Linear(in_features=1792, out_features=625),  # First linear layer
    nn.ReLU(),  # ReLU activation function
    nn.Dropout(p=0.3),  # Dropout layer with a dropout probability of 0.3
    nn.Linear(in_features=625, out_features=256),  # Second linear layer
    nn.ReLU(),  # ReLU activation function
    nn.Linear(in_features=256, out_features=2)  # Final linear layer with 2 outputs for binary classification
)

c:\Users\jxs75\Documents\GitHub\Pneumonia Classification\Pneumonia-Classification\.venv\lib\site-packages\timm\models\_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


In [5]:
# Load DenseNet-201
model_densenet = timm.create_model('densenet201', pretrained=False)
# Adjust the classifier layer of DenseNet-201 according to your project
model_densenet.classifier = nn.Sequential(
    nn.Linear(1920, 625),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(625, 256),
    nn.ReLU(),
    nn.Linear(256, 2)  # Assuming binary classification
)

In [6]:
# Load the trained weights
model_efficientnet.load_state_dict(torch.load('PneumoniaModel.pth'))
model_densenet.load_state_dict(torch.load('DenseNetPneumoniaModel.pth'))

<All keys matched successfully>

In [7]:
# Set both models to evaluation mode
model_efficientnet.to(device).eval()
model_densenet.to(device).eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNormAct2d(
      64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): ReLU(inplace=True)
    )
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): DenseBlock(
      (denselayer1): DenseLayer(
        (norm1): BatchNormAct2d(
          64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNormAct2d(
          128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): ReLU(inplace=True)
        )
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
  

In [8]:
# Define transformations for the test dataset
test_transform = T.Compose([
    T.Resize(size=(224, 224)),  # Resize images to the specified size
    T.ToTensor(),  # Convert images to PyTorch tensors
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize images using the same mean and std
])

In [9]:
from torch.utils.data import DataLoader 

test_dataset = datasets.ImageFolder(root='./chest_xray_data/test', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)  # Adjust batch_size as needed

In [10]:
# Soft Voting
def soft_voting_with_labels(model1, model2, loader):
    model1.to(device).eval()
    model2.to(device).eval()
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs1 = torch.softmax(model1(images), dim=1)
            outputs2 = torch.softmax(model2(images), dim=1)

            avg_outputs = (outputs1 + outputs2) / 2
            _, predicted_classes = torch.max(avg_outputs, 1)

            all_predictions.extend(predicted_classes.cpu().numpy())
            all_labels.extend(labels.numpy())

    return all_predictions, all_labels


In [11]:
# Get predictions
predictions, true_labels = soft_voting_with_labels(model_efficientnet, model_densenet, test_loader)

In [12]:
accuracy = np.mean(np.array(predictions) == np.array(true_labels))
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 89.42%
